## 텍스트 전처리 및 시각화 연습문제

#### 주어진 뉴스기사 데이터를 파악하고, 필요한 컬럼에 대하여 전처리하여 분석하시오.
- 데이터 파일 : 'kor_news_240326.xlsx'

### 1. 필요한 모듈 임포트

In [1]:
import pandas as pd
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus']=False

if platform.system() == 'Darwin':  # Mac
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # Windows
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~')

### 2. 데이터 읽고 파악하기

In [3]:
news = pd.read_excel('data/kor_news_240326.xlsx')
news.head(2)

,식별자,언론사,기고자,제목,분류,본문,URL
0,1,디지털타임스,박순원,"DL이앤씨, 올 매출목표 8.9조 성장 수익성 두 토끼 잡는다",경제>금융_재테크,"DL이앤씨, 안정적 재무구조 바탕으로 수익성 중심 내실경영\n\n\n\n'24년 매...",http://www.dt.co.kr/contents.html?article_no=2...
1,2,디지털타임스,박순원,"한신공영, 정선 임계~동해 신흥 도로건설공사 수주",지역>광주,한신공영이 1373억원 규모의 '국도42호선 정선 임계~동해 신흥 도로건설공사'를 ...,http://www.dt.co.kr/contents.html?article_no=2...


In [4]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19425 entries, 0 to 19424
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   식별자     19425 non-null  int64 
 1   언론사     19425 non-null  object
 2   기고자     18452 non-null  object
 3   제목      19425 non-null  object
 4   분류      19425 non-null  object
 5   본문      19425 non-null  object
 6   URL     18091 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### 3. 분야별로 뉴스 데이터 구분하기

- '분류' 컬럼을 정제하고 전처리하여 뉴스 분류 키워드들(예. 경제, 사회, 문화, 스포츠 등)을 추출하고,
- 분류한 키워드를 기준으로 각 뉴스를 구분하는 '구분' 컬럼을 생성

In [5]:
news.분류.values

array(['경제>금융_재테크', '지역>광주', '경제>금융_재테크', ..., '사회>노동_복지', '경제>산업_기업',
       '사회>장애인'], dtype=object)

In [6]:
news['분류리스트'] = news.분류.str.split('>')
news['분류리스트']

0            [경제, 금융_재테크]
1                [지역, 광주]
2            [경제, 금융_재테크]
3                [사회, 여성]
4        [스포츠, 올림픽_아시안게임]
               ...       
19420           [스포츠, 축구]
19421    [스포츠, 올림픽_아시안게임]
19422         [사회, 노동_복지]
19423         [경제, 산업_기업]
19424           [사회, 장애인]
Name: 분류리스트, Length: 19425, dtype: object

In [7]:
news['구분'] = news.분류리스트.str[0]
news['중분류'] = news.분류리스트.str[1]
news['소분류'] = news.분류리스트.str[2]

In [8]:
news.구분.value_counts()

구분
지역       4974
경제       4262
사회       2618
정치       2365
문화       1914
IT_과학    1121
스포츠       889
국제        843
미분류       439
Name: count, dtype: int64

### 4. '제목' 컬럼 데이터 전처리 및 시각화

- 제목 컬럼 데이터를 전처리하고
- 빈도가 높은 키워드를 추출하여
- 막대그래프와 워드클라우드로 시각화

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
news.제목

0                    DL이앤씨, 올 매출목표 8.9조 성장 수익성 두 토끼 잡는다
1                           한신공영, 정선 임계~동해 신흥 도로건설공사 수주
2                            기후카드 충전 편해져요 27일부터 신용카드 가능
3                       "낳기만 하면 지자체가 키운다" 장기 지원책 꺼내든 전남
4        송범진 크리에이터 “배드민턴이 대한민국 최고 인기 스포츠가 되길” [S크리에이터①]
                              ...                      
19420                      제1회 익산시체육회장배 동호인 한마음 축구대회 개막
19421                       서남용 완주군의장, 완주군청 근대5종 선수단 격려
19422       [정년 연장의 '빛과 그림자'] (하)대안은 있나 - 노사정 대타협이 '열쇠'
19423                           정년연장 이젠 정부가 나서야 할 당면 과제
19424                인천시 동구, '지역사회서비스 투자사업' 이용자 312명 모집
Name: 제목, Length: 19425, dtype: object

In [11]:
news_title = list(news['제목'].values)

In [12]:
tokens_news = [okt.nouns(title) for title in news_title]
tokens_title = sum(tokens_news, [])

In [13]:
# 한글자 제거
tokens_title2 = [token for token in tokens_title if len(token) > 1]
tokens_title2

['매출',
 '목표',
 '성장',
 '수익',
 '토끼',
 '공영',
 '정선',
 '임계',
 '동해',
 '신흥',
 '도로',
 '건설',
 '공사',
 '수주',
 '기후',
 '카드',
 '충전',
 '신용카드',
 '가능',
 '지자체',
 '장기',
 '지원',
 '전남',
 '송범',
 '크리에이터',
 '배드민턴',
 '대한민국',
 '최고',
 '인기',
 '스포츠',
 '크리에이터',
 '희망',
 '상가',
 '공급',
 '청년',
 '대상',
 '송영',
 '한미',
 '회장',
 '아들',
 '후계',
 '장녀',
 '임주현',
 '분양',
 '신도시',
 '사비',
 '폭탄',
 '여배우',
 '대세',
 '박지현',
 '김지원',
 '돌파구',
 '에듀',
 '테크',
 '새봄',
 '웅진',
 '대표',
 '실적',
 '반등',
 '몰카',
 '오히려',
 '증거',
 '무혐의',
 '물뽕',
 '마약',
 '사건',
 '전말',
 '디펜딩',
 '챔피언',
 '변상일',
 '연패',
 '조준',
 '춘란배',
 '강전',
 '유창혁',
 '연패',
 '시동',
 '백전노장',
 '승부',
 '대주배',
 '개막',
 '왕세자',
 '비도',
 '투병',
 '고백',
 '영국',
 '이하',
 '환자',
 '폭증',
 '미스터리',
 '민주당',
 '지역구',
 '후보자',
 '라인',
 '전수',
 '조사',
 '친윤',
 '불패',
 '국민',
 '지역구',
 '후보자',
 '라인',
 '전수',
 '조사',
 '노종',
 '화의',
 '기업',
 '파인더',
 '동학',
 '개미',
 '주주',
 '제안',
 '문턱',
 '넘사벽',
 '단독',
 '클라우드',
 '보안',
 '기준',
 '울산',
 '중구',
 '가족',
 '센터',
 '그루',
 '부방',
 '한국어',
 '부모',
 '대신',
 '기초',
 '학력',
 '평균',
 '버는데',
 '서울',
 '살수',
 '아파트',
 '채중',
 '중도',
 '초

In [14]:
from collections import Counter

In [15]:
# 데이터프레임으로 변환 함수

def token_to_df(tokens_title2):
    title_cnt = Counter(tokens_title2)
    df_title_cnt = pd.DataFrame(pd.Series(title_cnt), columns=['Freq'])
    sorted_title_cnt = df_title_cnt.sort_values(by='Freq', ascending=False)
    return sorted_title_cnt

token_to_df(tokens_title2)

,Freq
포토,753
지원,631
사업,622
총선,587
개최,534
...,...
수책,1
극우,1
유대,1
집밥,1


In [16]:
# 빈도 막대그래프 함수
def top_word_barplot(cnt_df, max_words=10):
    df = cnt_df.iloc[:max_words]
    sns.barplot(data=df, x='Freq', y=df.index)
    plt.title(f'뉴스 제목 Top{max_words} 단어 빈도그래프')
    plt.show()

top_word_barplot(sorted_title_cnt, 15)

NameError: name 'sorted_title_cnt' is not defined

In [ ]:
# 워드클라우드 시각화
from wordcloud import WordCloud

font_path = 'C:/windows/Fonts/malgun.ttf'
wordcloud = WordCloud(font_path=font_path,
                      max_words=50,
                     ).generate_from_frequencies(title_cnt)
plt.axis('off')
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

### 5. '본문' 컬럼 데이터 전처리 및 시각화

- 본문 컬럼 데이터를 전처리하고
- 빈도가 높은 키워드를 추출하여(구분/분야별로)
- 막대그래프와 워드클라우드로 시각화

In [ ]:
news.본문

In [ ]:
news_main = list(news['본문'].values)
tokens_news = [okt.nouns(main) for main in news_main]

In [ ]:
tokens_main = sum(tokens_news, [])

In [ ]:
tokens_main2 = [token for token in tokens_main if len(token) > 1]
tokens_main2

In [ ]:
# 데이터프레임으로 변환
sorted_main_cnt = token_to_df(tokens_main2)

In [ ]:
# 빈도 막대그래프
top_word_barplot(sorted_main_cnt, 15)

In [ ]:
# 워드클라우드 시각화
from wordcloud import WordCloud

main_cnt = Counter(tokens_main2)
font_path = 'C:/windows/Fonts/malgun.ttf'
wordcloud = WordCloud(font_path=font_path,
                      max_words=50,
                     ).generate_from_frequencies(main_cnt)
plt.axis('off')
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

----